In [39]:
import sympy as sy
from sympy import pi, cos, sin, tan
from IPython.display import display

#q1, q2, q3, q4 = sy.symbols("q1, q2, q3, q4")  # 関節角度
t = sy.Symbol("t")
q1 = sy.Function("q1")
q2 = sy.Function("q2")
q3 = sy.Function("q3")
q4 = sy.Function("q4")
l1, l2, l3, l4 = sy.symbols("l1, l2, l3, l4")  # リンク長さ
lg1, lg2, lg3, lg4 = sy.symbols("lg1, lg2, lg3, lg4")  # 重心までの長さ
m1, m2, m3, m4 = sy.symbols("m1, m2, m3, m4")  # 質量
Ig1, Ig2, Ig3, Ig4 = sy.symbols("I1, I2, I3, I4")  # 慣性モーメント
g = sy.Symbol("g")  # 重力加速度

def R(q):
    return sy.Matrix([
        [cos(q), -sin(q)],
        [sin(q), cos(q)],
    ])


def HTM(q, x, y):
    return sy.Matrix([
        [cos(q), -sin(q), x],
        [sin(q), cos(q), y],
        [0, 0, 1],
    ])


# 重心の位置
xg1 = R(q1(t)) * sy.Matrix([[lg1, 0]]).T
xg2 = R(q1(t)) * sy.Matrix([[l1, 0]]).T + \
    R(q1(t) + q2(t)) * sy.Matrix([[lg2, 0]]).T
xg3 = R(q1(t)) * sy.Matrix([[l1, 0]]).T +\
    R(q1(t) + q2(t)) * sy.Matrix([[l2, 0]]).T +\
        R(q1(t) + q2(t) + q3(t)) * sy.Matrix([[lg3, 0]]).T
xg4 = R(q1(t)) * sy.Matrix([[l1, 0]]).T +\
    R(q1(t) + q2(t)) * sy.Matrix([[l2, 0]]).T +\
        R(q1(t) + q2(t) + q3(t)) * sy.Matrix([[l3, 0]]).T +\
            R(q1(t) + q2(t) + q3(t) + q4(t)) * sy.Matrix([[lg4, 0]]).T

# 重心位置の時間微分
xg1_dot = sy.diff(xg1, t)
xg2_dot = sy.diff(xg2, t)
xg3_dot = sy.diff(xg3, t)
xg4_dot = sy.diff(xg4, t)


# 運動エネルギー
T1 = 1/2*m1*xg1_dot.norm()**2 + 1/2*Ig1*sy.Derivative(q1(t),t)**2
T2 = 1/2*m2*xg2_dot.norm()**2 +\
    1/2*Ig2*(sy.Derivative(q1(t),t) + sy.Derivative(q2(t),t))**2
T3 = 1/2*m3*xg3_dot.norm()**2 +\
    1/2*Ig3*(sy.Derivative(q1(t),t) + sy.Derivative(q2(t),t) + sy.Derivative(q3(t),t))**2
T4 = 1/2*m4*xg4_dot.norm()**2 +\
    1/2*Ig4*(sy.Derivative(q1(t),t) + sy.Derivative(q2(t),t) + sy.Derivative(q3(t),t) + sy.Derivative(q4(t),t))**2

# 位置エネルギー
U1 = m1*g*xg1[1,0]
U2 = m2*g*xg2[1,0]
U3 = m3*g*xg3[1,0]
U4 = m4*g*xg4[1,0]

# ラグランジアン
L = (T1 + T2 + T3 + T4) - (U1 + U2 + U3 + U4)

L

0.5*I1*Derivative(q1(t), t)**2 + 0.5*I2*(Derivative(q1(t), t) + Derivative(q2(t), t))**2 + 0.5*I3*(Derivative(q1(t), t) + Derivative(q2(t), t) + Derivative(q3(t), t))**2 + 0.5*I4*(Derivative(q1(t), t) + Derivative(q2(t), t) + Derivative(q3(t), t) + Derivative(q4(t), t))**2 - g*lg1*m1*sin(q1(t)) - g*m2*(l1*sin(q1(t)) + lg2*sin(q1(t) + q2(t))) - g*m3*(l1*sin(q1(t)) + l2*sin(q1(t) + q2(t)) + lg3*sin(q1(t) + q2(t) + q3(t))) - g*m4*(l1*sin(q1(t)) + l2*sin(q1(t) + q2(t)) + l3*sin(q1(t) + q2(t) + q3(t)) + lg4*sin(q1(t) + q2(t) + q3(t) + q4(t))) + 0.5*m1*(Abs(lg1*sin(q1(t))*Derivative(q1(t), t))**2 + Abs(lg1*cos(q1(t))*Derivative(q1(t), t))**2) + 0.5*m2*(Abs(l1*sin(q1(t))*Derivative(q1(t), t) + lg2*(Derivative(q1(t), t) + Derivative(q2(t), t))*sin(q1(t) + q2(t)))**2 + Abs(l1*cos(q1(t))*Derivative(q1(t), t) + lg2*(Derivative(q1(t), t) + Derivative(q2(t), t))*cos(q1(t) + q2(t)))**2) + 0.5*m3*(Abs(l1*sin(q1(t))*Derivative(q1(t), t) + l2*(Derivative(q1(t), t) + Derivative(q2(t), t))*sin(q1(t) + q2

In [41]:
dLdq1_dot = sy.diff(L, sy.Derivative(q1(t),t))
dLdq2_dot = sy.diff(L, sy.Derivative(q2(t),t))
dLdq3_dot = sy.diff(L, sy.Derivative(q3(t),t))
dLdq4_dot = sy.diff(L, sy.Derivative(q4(t),t))

DdLdq1_dotDt = sy.diff(dLdq1_dot, t)
DdLdq2_dotDt = sy.diff(dLdq2_dot, t)
DdLdq3_dotDt = sy.diff(dLdq3_dot, t)
DdLdq4_dotDt = sy.diff(dLdq4_dot, t)

dLdq1 = sy.diff(L, q1(t))
dLdq2 = sy.diff(L, q2(t))
dLdq3 = sy.diff(L, q3(t))
dLdq4 = sy.diff(L, q4(t))

In [42]:
u1 = DdLdq1_dotDt - dLdq1
u2 = DdLdq2_dotDt - dLdq2
u3 = DdLdq3_dotDt - dLdq3
u4 = DdLdq4_dotDt - dLdq4

In [ ]:
hoge = sy.diff(xg1, t)
sy.diff(hoge, t)

Matrix([
[-lg1*sin(q1(t))*Derivative(q1(t), (t, 2)) - lg1*cos(q1(t))*Derivative(q1(t), t)**2],
[-lg1*sin(q1(t))*Derivative(q1(t), t)**2 + lg1*cos(q1(t))*Derivative(q1(t), (t, 2))]])

In [43]:
u1

1.0*I1*Derivative(q1(t), (t, 2)) + 0.5*I2*(2*Derivative(q1(t), (t, 2)) + 2*Derivative(q2(t), (t, 2))) + 0.5*I3*(2*Derivative(q1(t), (t, 2)) + 2*Derivative(q2(t), (t, 2)) + 2*Derivative(q3(t), (t, 2))) + 0.5*I4*(2*Derivative(q1(t), (t, 2)) + 2*Derivative(q2(t), (t, 2)) + 2*Derivative(q3(t), (t, 2)) + 2*Derivative(q4(t), (t, 2))) + g*lg1*m1*cos(q1(t)) + g*m2*(l1*cos(q1(t)) + lg2*cos(q1(t) + q2(t))) + g*m3*(l1*cos(q1(t)) + l2*cos(q1(t) + q2(t)) + lg3*cos(q1(t) + q2(t) + q3(t))) + g*m4*(l1*cos(q1(t)) + l2*cos(q1(t) + q2(t)) + l3*cos(q1(t) + q2(t) + q3(t)) + lg4*cos(q1(t) + q2(t) + q3(t) + q4(t))) - 0.5*m1*(2*(re(lg1*sin(q1(t))*Derivative(q1(t), t))*Derivative(re(lg1*sin(q1(t))*Derivative(q1(t), t)), q1(t)) + im(lg1*sin(q1(t))*Derivative(q1(t), t))*Derivative(im(lg1*sin(q1(t))*Derivative(q1(t), t)), q1(t)))*Abs(lg1*sin(q1(t))*Derivative(q1(t), t))*sign(lg1*sin(q1(t))*Derivative(q1(t), t))/(lg1*sin(q1(t))*Derivative(q1(t), t)) + 2*(re(lg1*cos(q1(t))*Derivative(q1(t), t))*Derivative(re(lg1*co